# Video Processing for BT Lab
![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import re

In [2]:
repo_dir = os.getcwd()

In [3]:
def extract_info_from_filepath(filepaths:list[str]) -> list[dict]:
    all_videos = []
    file_info_from_path = {
        0 : "patient id",
        1 : "distance",
        2 : "blanket",
        3 : "breathing",
        4 : "filename"
    }

    splice_stop = 5

    for filepath in filepaths:
        filepath_no_backslash = filepath.replace("\\", "/")
        filepath_to_list = filepath_no_backslash.split("/")
        filepath_to_list = filepath_to_list[-splice_stop:]
        patient_info = {tag:filepath_to_list[key] for key, tag in file_info_from_path.items() if key<splice_stop}
        all_videos.append(patient_info)

    return all_videos

## Extract information from the exported log files and create containers for the metadata (stage 1.2)

In [4]:
records_folder = repo_dir + "/record" 

In [5]:
"""
    Read log files
"""

os.chdir(records_folder)
log_files = [dir_file for dir_file in os.listdir() if dir_file.endswith(".log")]
video_files = []

for log_file in log_files:
    try:
        with open(log_file, "r") as log_file_data:
            log_read = log_file_data.read().split("\n")
            video_files.append(log_read)
    except FileNotFoundError as e:
        print(type(e))

In [6]:
"""
    make dictionary out of filenames (rgb)
"""
test_list = [video_files[0][0]]
metadata_rgb = extract_info_from_filepath(test_list)
metadata_rgb

[{'patient id': 'Arun',
  'distance': '2 Meters',
  'blanket': 'With Blankets',
  'breathing': 'Hold Breath',
  'filename': 'Arun2.mp4'}]

In [7]:
"""
    make dictionary out of filenames(thermal)
"""
test_list = [video_files[1][0]]
metadata_thermal = extract_info_from_filepath(test_list)
metadata_thermal

[{'patient id': '',
  'distance': '1',
  'blanket': '2 meters',
  'breathing': 'H',
  'filename': '1_H_WOB_2m_2.avi'}]

In [9]:
test_list = [video_files[1][0]]
test_list

['E:/Christian/DI_centre_structured/DI_THERMAL_FLIR//1\\2 meters\\H\\1_H_WOB_2m_2.avi']